### 抽取出兩個長文本欄位，組合至新的 Excel

In [1]:
import pandas as pd
# df = pd.read_excel('104_data.xlsx')
# # 將兩個欄位組合成新的 DataFrame
# new_df = pd.concat([job_content, other_requirements], axis=1)
# # 匯出成新的 Excel 檔案
# new_df.to_excel('textual_analysis.xlsx', index=False)

In [2]:
file_path = 'textual_analysis.xlsx'
data = pd.read_excel(file_path)

data.head()

,職務內容,其他條件
0,職位描述：\n\n我們正在尋找有經驗的軟體工程師加入開發團隊，理想候選人需至少有 2 年以上...,NaN
1,主要負責API程式與網路平台串接及開發EXCEL巨集或程式以整合跨系統/應用軟體作業流程。\...,NaN
2,Responsibilities\n1. Add new features to our d...,1. Interested in developing the novel document...
3,工作內容：\n\n1. 通過.NET進行移動機器人平台開發\n2. 通過.NET/C++進行...,"具有微服務架構經驗,機器人相關演算法經驗或生產物流設計規劃經驗者尤佳"
4,我們在尋找一位軟體工程師，維護及優化公司現有的多層次傳銷系統，並配合公司的需求開發新功能；此...,熟悉多層次傳銷系統者佳


In [3]:
job_content = data['職務內容']
# 選擇「其他條件」欄位，並新增到新的 DataFrame 中
other_requirements = data['其他條件']

job_content

0       職位描述：\n\n我們正在尋找有經驗的軟體工程師加入開發團隊，理想候選人需至少有 2 年以上...
1       主要負責API程式與網路平台串接及開發EXCEL巨集或程式以整合跨系統/應用軟體作業流程。\...
2       Responsibilities\n1. Add new features to our d...
3       工作內容：\n\n1. 通過.NET進行移動機器人平台開發\n2. 通過.NET/C++進行...
4       我們在尋找一位軟體工程師，維護及優化公司現有的多層次傳銷系統，並配合公司的需求開發新功能；此...
                              ...                        
2352    Job Description (職務說明)：\nWorks with the global...
2353    •\tImplement core features in mobile games_x00...
2354    1.購物網站建置與開發\n2.會員活動串接與規劃\n3.報表需求確認與開發\n4.提升平台效...
2355    1.程式設計 (IBM AS400 RPG )、 (VB)、(C#)、 ( ASP.net)...
2356    【工作內容】 \n- 負責APP開發\n- 優化體驗及效能\n \n【工作技能】 \n- 熟...
Name: 職務內容, Length: 2357, dtype: object

In [4]:
# 將職務內容 以及 其他條件 欄位的兩千多筆資料 分別成一個字串
job_content_text = data['職務內容'].fillna('').astype(str).str.cat(sep=' ')
other_requirements_text = data['其他條件'].fillna('').astype(str).str.cat(sep=' ')

# job_content_text

### 斷詞

## TF-IDF 加權

#### 測試停用詞 文檔是否有用

In [5]:
with open('jieba_dict/stop_words.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f]
stopwords

['the',
 'to',
 'and',
 'x000d',
 'to',
 'of',
 'is',
 'and',
 'to',
 'in',
 'that',
 'we',
 'for',
 'an',
 'are',
 'by',
 'be',
 'as',
 'on',
 'with',
 'can',
 'if',
 'from',
 'which',
 'you',
 'it',
 'this',
 'then',
 'at',
 'have',
 'all',
 'not',
 'one',
 'has',
 'or',
 'that',
 '的',
 '了',
 '和',
 '是',
 '就',
 '都',
 '而',
 '及',
 '與',
 '著',
 '或',
 '一個',
 '沒有',
 '我們',
 '你們',
 '妳們',
 '他們',
 '她們',
 '是否']

### 計算「職缺內容」的 TF-IDF 值

In [6]:
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer

# 讀取停用詞檔案
with open('jieba_dict/stop_words.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f]

# 移除停用詞後再進行斷詞
filtered_text = [word for word in jieba.cut(job_content_text) if word not in stopwords]

# 將斷詞後的結果轉換為字符串
text = ' '.join(filtered_text)

# 定義TF-IDF向量化器，不使用停用詞
tfidf_vectorizer = TfidfVectorizer()

# 將文本資料轉換為TF-IDF向量
tfidf_matrix = tfidf_vectorizer.fit_transform([text])

# 取得特徵詞列表
feature_names = tfidf_vectorizer.get_feature_names_out()

# 取得詞彙與權重的對應關係
word_weight = dict(zip(feature_names, tfidf_matrix.toarray()[0]))

# 依據TF-IDF值對詞彙進行排序
sorted_word_weight = sorted(word_weight.items(), key=lambda x: x[1], reverse=True)

# 取得前40項關鍵詞
top_keywords = sorted_word_weight[:40]

# 列印前40項關鍵詞
for word, weight in top_keywords:
    print(word, weight)

ModuleNotFoundError: No module named 'jieba'

### 職缺內容的 TF、IDF和 TF-IDF值

In [ ]:
for word in set(filtered_text):
    df[word] = sum(1 for doc in filtered_text if word in doc)
    print(f"詞彙 {word} 的 IDF 值為: {df[word]}")

In [2]:
import jieba
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import math

# 讀取停用詞檔案
with open('jieba_dict/stop_words.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f]

# 移除停用詞後再進行斷詞
filtered_text = [word for word in jieba.cut(job_content_text) if word not in stopwords and len(word) > 1 and not re.match(r"[\d]", word)]

# 計算詞頻
tf = {}
for word in filtered_text:
    tf[word] = tf.get(word, 0) + 1
total_words = len(filtered_text)
for word in tf:
    tf[word] /= total_words

# 計算文檔數目
total_docs = len(filtered_text)  # 請替換 documents 為你的文檔列表，其中每個元素都是一個文檔


# 計算包含每個詞彙的文檔數目
df = {}
for word in set(filtered_text):
    df[word] = sum(1 for doc in filtered_text if word in doc)
    # print(f"詞彙 {word} 的 DF 值為: {df[word]}")

# 計算 IDF
idf = {}
for word in df:
    idf[word] = math.log(total_docs / (df[word] + 1))
    # print(f"詞彙 {word} 的 IDF 值為: {idf[word]}")

# 計算 TF-IDF
tfidf = {}
for word in tf:
    tfidf[word] = tf[word] * idf[word]

# 依照 TF-IDF 值排序
sorted_tfidf = sorted(tfidf.items(), key=lambda x: x[1], reverse=True)

# 取得前40項關鍵詞及其對應的 TF、IDF、TF-IDF 值
top_keywords = sorted_tfidf[:40]

# 列印前40項關鍵詞及其對應的 TF、IDF、TF-IDF 值
for word, tfidf_value in top_keywords:
    tf_value = tf[word]
    idf_value = idf[word]
    print(f"詞彙: {word}, TF: {tf_value}, IDF: {idf_value}, TF-IDF: {tfidf_value}")

ModuleNotFoundError: No module named 'jieba'

### 其他條件 的 TF、IDF和 TF-IDF值

In [1]:
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer

# 讀取停用詞檔案
with open('jieba_dict/stop_words.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f]

# 移除停用詞後再進行斷詞
filtered_text = [word for word in jieba.cut(job_content_text) if word not in stopwords]

# 將斷詞後的結果轉換為字符串
text = ' '.join(filtered_text)

# 定義TF-IDF向量化器，不使用停用詞
tfidf_vectorizer = TfidfVectorizer()

# 將文本資料轉換為TF-IDF向量
tfidf_matrix = tfidf_vectorizer.fit_transform([text])

# 取得特徵詞列表
feature_names = tfidf_vectorizer.get_feature_names_out()

# 取得詞彙與權重的對應關係
word_weight = dict(zip(feature_names, tfidf_matrix.toarray()[0]))

# 依據TF-IDF值對詞彙進行排序
sorted_word_weight = sorted(word_weight.items(), key=lambda x: x[1], reverse=True)

# 取得前40項關鍵詞
top_keywords = sorted_word_weight[:40]

# 列印前40項關鍵詞
for word, weight in top_keywords:
    print(word, weight)

ModuleNotFoundError: No module named 'jieba'